In [8]:
from BigDFT import Datasets as D, Calculators as C, Inputfiles as I, Logfiles as lf
from BigDFT.Database import Molecules
from futile.Utils import write
import numpy as np
import matplotlib.pyplot as plt
import os, sys
sys.path.insert(0,'../')
import StatPol as SP

## Computation of the statical polarizability for a single study (of the nsp type)

Discuss the workflow to compute the statical polarizability of a specific study defined by the molecule type, the xc and the psp. We choose, for instance

In [9]:
molecule = 'CO'
std = ('lda', 'hgh-k')

Set some global parameters

In [10]:
wf_convergence = 1.0e-6
hgrids = 0.3
rmult_fine = 9.0

Build the folder associated to the molecule and move there

In [11]:
if not os.path.isdir(molecule): os.mkdir(molecule)
os.chdir(molecule)
sys.path.insert(1,'../../')

Build the folder associated to the study and define the path variable

In [16]:
path=std[0]+'-'+std[1]
if not os.path.isdir(path): os.mkdir(path)
path

'lda-hgh-k'

Build a dictionary to collect the results of the analysis

In [17]:
results = {}

In [18]:
# read the posinp from the bigdft database
posinp=Molecules.Molecule(molecule)
print posinp

{'units': 'angstroem', 'positions': [{'C': [0.0, 0.0, 0.0], 'sym': 'C'}, {'sym': 'O', 'O': [0.0, 0.0, 1.1282]}], 'global monopole': 0.0}


In [19]:
code=C.SystemCalculator(omp=2,mpi_run='mpirun -np 4',skip=True)

Initialize a Calculator with OMP_NUM_THREADS=2 and command mpirun -np 4 /home/marco/Applications/BigDFT/binaries/v1.8.3/install/bin/bigdft


Perform a convergence study for the gs. Use the value of the total energy (or the value of the gs dipole) to set the dimension of the box

TO DO : modify this procedure to include the pbe0 and nlcc psp's

In [21]:
reload(SP)
rtol=1e-3 #relative tolerance for the gs convergence using the total energy as control quantity

inp = I.Inputfile()
inp.set_hgrid(hgrids)
inp.set_xc(std[0].upper())
inp.set_wavefunction_convergence(wf_convergence)

rmult_coarse = [1.0*i for i in range(3,12)]
data = []
code.update_global_options(verbose=False)
for r in rmult_coarse:
    gs_study = D.Dataset(label=molecule+'_GS',run_dir=path,posinp=posinp)
    gs_study.set_postprocessing_function(SP.get_energy)
    #gs_study.set_postprocessing_function(SP-get_dipole)
    inp.set_rmult(coarse=r,fine=rmult_fine)
    idd={'rmult':r}
    gs_study.append_run(id=idd,runner=code,input=inp)
    data.append(gs_study)
    
results['gs_conv'] = SP.seek_convergence(rt=rtol,label='rmult',values=rmult_coarse,data=data)
results

Perform the run with rmult 3.0
Perform the run with rmult 4.0
Convergence for rmult 3.0 failed
Perform the run with rmult 5.0
Convergence achieved for rmult 4.0


{'gs_conv': {'converged': True,
  'converged_value': 4.0,
  'label': 'rmult',
  'results': {3.0: -21.6222251436458,
   4.0: -21.66069624514632,
   5.0: -21.663445020771178,
   6.0: None,
   7.0: None,
   8.0: None,
   9.0: None,
   10.0: None,
   11.0: None},
  'values': [3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0]}}

It emerges that the convergence performed with the energy is less demanding that the one done with the dipole!

From this analysis we extract the converged log of the gs to be used as the starting point for the computation of the statical polarizability

In [22]:
conv_val = results['gs_conv']['converged_value']
gslog = 'log-'+data[rmult_coarse.index(conv_val)].names[0]+'.yaml'
print gslog
gs = lf.Logfile(path+os.sep+gslog)

log-rmult:4.0.yaml


Convergence flow for the calculation of alpha: take the rmult of the gs convergence. Perform the field convergence with this value of rmult and finally perform the rmult convergence for alpha using the converged value of the field

In [25]:
rtol=1e-2 #relative tolerance for the alpha convergence

inp = I.Inputfile()
inp.set_hgrid(gs.log['dft']['hgrids'])
inp.set_xc(std[0].upper())
inp.set_wavefunction_convergence(gnrm=wf_convergence)
inp.set_rmult(gs.log['dft']['rmult'])
print inp

results['field_conv']=SP.perform_field_convergence(rt=rtol,run_dir=path,input=inp,\
                      runner=code,posinp=posinp,ppf=SP.eval_alpha)
print ''

f=results['field_conv']['converged_value']
rmult_list=SP.build_rmult_list(gs)
results['rmult_conv']=SP.perform_rmult_convergence(rt=rtol,run_dir=path,intensity=f,\
                      rmult=rmult_list,input=inp,runner=code,posinp=posinp,ppf=SP.eval_alpha)

{'dft': {'ixc': 'LDA', 'rmult': [4.0, 9.0], 'hgrids': 0.3, 'gnrm_cv': 1e-06}}
Perform the run with field_int 0.01
Perform the run with field_int 0.005
Convergence achieved for field_int 0.01

Perform the run with rmult 4.0
Perform the run with rmult 5.0
Convergence for rmult 4.0 failed
Perform the run with rmult 6.0
Convergence for rmult 5.0 failed
Perform the run with rmult 7.0
Convergence achieved for rmult 6.0


In [27]:
r_conv = results['rmult_conv']['converged_value']
alpha_final = results['rmult_conv']['results'][r_conv]
print alpha_final
print SP.eval_alpha_avg(alpha_final)

[[ 1.26353100e+01 -1.39015000e-05 -5.72575000e-05]
 [-1.39015000e-05  1.26353100e+01 -5.72575000e-05]
 [-3.75000000e-05 -3.75000000e-05  1.58602435e+01]]
13.710287833333332


In [28]:
results

{'field_conv': {'converged': True,
  'converged_value': 0.01,
  'label': 'field_int',
  'results': {0.0005: None,
   0.001: None,
   0.005: matrix([[ 1.1710982e+01, -2.4880000e-04,  6.0950000e-04],
           [-2.4880000e-04,  1.1710982e+01,  6.0950000e-04],
           [ 5.8000000e-04,  5.8000000e-04,  1.4605729e+01]]),
   0.01: matrix([[ 1.1718940e+01, -2.4485000e-04,  5.6675000e-04],
           [-2.4485000e-04,  1.1718940e+01,  5.6675000e-04],
           [ 5.7000000e-04,  5.7000000e-04,  1.4614152e+01]])},
  'values': [0.01, 0.005, 0.001, 0.0005]},
 'gs_conv': {'converged': True,
  'converged_value': 4.0,
  'label': 'rmult',
  'results': {3.0: -21.6222251436458,
   4.0: -21.66069624514632,
   5.0: -21.663445020771178,
   6.0: None,
   7.0: None,
   8.0: None,
   9.0: None,
   10.0: None,
   11.0: None},
  'values': [3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0]},
 'rmult_conv': {'converged': True,
  'converged_value': 6.0,
  'label': 'rmult',
  'results': {4.0: matrix([[ 1.1718940e+

This workflow is translated into a single method and stored in the file 'nsp_workflow.py'